# Working

In [5]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("src/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:

from src.data import get_track , getBatch
from src.model import Transformer
import glob
import numpy as np

In [11]:
aPaths = glob.glob("data/*AllPSA.npy")
hpaths = glob.glob("data/*.hits.npy")
ap = np.random.choice(aPaths)
hp = np.random.choice(hpaths)
trk =  get_track(apth=ap,hpth=hp)
batch = getBatch(trk,batch_size=32)
X = batch[:,:,2:5]
Y = batch[:,:,5:]
model = Transformer()

# Experiment

In [43]:
predictions, loss = model(X,Y)

In [44]:
loss

<tf.Tensor: shape=(), dtype=float32, numpy=249.33801>